Code for LSTM<br>
Authors: Rajat and Nima

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dropout, Dense, Input, LSTM, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model,  model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split

import sys, os, re, csv, codecs
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


import gensim.models.keyedvectors as word2vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import  h5py

Using TensorFlow backend.


In [2]:
# here you need to specify filepath for the dataset
PATH = "/home/ubuntu/fastai/courses/Ait590/"
train = pd.read_csv(PATH+"Toxic_PreProc.csv")

In [7]:
!ls /home/ubuntu/fastai/courses/Ait590/


Embedding	   proj-draft1.ipynb	  Untitled1.ipynb   Word2Vec.ipynb
Glove_Rajat.ipynb  Toxic_PreProc.csv	  Untitled.ipynb
model.bin	   Toxic_PreProc.csv.zip  Unzip_File.ipynb


In [3]:
list_classes = ["toxic", "severe_toxic", "obscene","threat", "insult", "identity_hate"]
y = train[list_classes].values
sentence_train = train["comment_text"].astype(str)
#sentence_test = test["comment_text"].astype(str)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(sentence_train, y, test_size=0.15, random_state=42)

In [5]:
#Choosing only 20000 frequent words
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)


In [6]:
#Tokenizing Sentence
tokenizer.fit_on_texts(list(X_train))
token_train = tokenizer.texts_to_sequences(X_train)
token_test = tokenizer.texts_to_sequences(X_test)

In [8]:
print token_train[:3]

[[97, 121, 458, 435, 3997, 11419, 4, 3, 167, 137, 45, 9, 1, 108, 19128], [2435, 13863, 602, 946, 875, 6745, 6370, 1271], [286, 3, 64, 3582, 1054, 48, 286, 18319, 3865, 3, 3541, 1054, 27, 177, 4, 3, 20]]


In [7]:
#Padding sentences to length 100
max_words = 100
X_train = pad_sequences(token_train, maxlen=max_words)
X_test = pad_sequences(token_test, maxlen=max_words)

## LSTM Fully Trained

In [19]:
inp = Input(shape=(max_words,))

In [21]:
#Embedding Layer
x = Embedding(max_features, 128)(inp)

In [23]:
#Running the bidirectional LSTM model. We are doing 10% regular/recurrent dropout  
x = LSTM(60,return_sequences=True, name="LSTM")(x)

In [24]:
#Adding the maxpooling layer
x = GlobalMaxPool1D()(x)

In [25]:
#One more dropout layer to help with any overfitting
x = Dropout(0.1)(x)

In [26]:
#passing it through a nonlinear (relu) layer
x = Dense(50, activation="relu")(x)

In [27]:
#One more dropout layer to help with any overfitting
x = Dropout(0.1)(x)

In [28]:
#this is the output layer. we need to use a nonlinear softmax or sigmoid for classification
# Since we have a multi label classification, we'll have to use sigmoid. softmax won't work.
x= Dense(6, activation="sigmoid")(x)

In [29]:
#Compiling our model
model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
LSTM (LSTM)                  (None, 100, 60)           45360     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 60)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                3050      
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
__________

In [31]:
#Fitting our model on dataset
batch_size = 32
epochs = 4
log = model.fit(X_train,y_train, batch_size=batch_size, validation_split=0.1, epochs=epochs)

Train on 122071 samples, validate on 13564 samples
Epoch 1/4
122071/122071 [==============================] - 563s 5ms/step - loss: 0.0762 - acc: 0.9764 - val_loss: 0.0531 - val_acc: 0.9801
Epoch 2/4
122071/122071 [==============================] - 557s 5ms/step - loss: 0.0461 - acc: 0.9829 - val_loss: 0.0505 - val_acc: 0.9811
Epoch 3/4
122071/122071 [==============================] - 558s 5ms/step - loss: 0.0396 - acc: 0.9847 - val_loss: 0.0511 - val_acc: 0.9806
Epoch 4/4
122071/122071 [==============================] - 557s 5ms/step - loss: 0.0340 - acc: 0.9866 - val_loss: 0.0547 - val_acc: 0.9806


In [32]:
import  h5py

In [37]:
model.save("LSTM_model_Best_weights.h5")

In [10]:
y_pred = loaded_model.predict(X_test, verbose=1)


23936/23936 [==============================] - 35s 1ms/step


In [41]:
#plotting the confusion matrix
print("Toxic:\n{}".format(confusion_matrix(y_test[:,0], y_pred[:,0])))
print("\nSevere Toxic: \n{}".format(confusion_matrix(y_test[:,1], y_pred[:,1])))
print("\nObscene: \n{}".format(confusion_matrix(y_test[:,2], y_pred[:,2])))
print("\nThreat: \n{}".format(confusion_matrix(y_test[:,3], y_pred[:,3])))
print("\nInsult: \n{}".format(confusion_matrix(y_test[:,4], y_pred[:,4])))
print("\nIdentity Hate: \n{}".format(confusion_matrix(y_test[:,5], y_pred[:,5])))
    

Toxic Confusion Matrixs: 
[[21272   404]
 [  539  1721]]

Severe Toxic: 
[[23646    65]
 [  162    63]]

Obscene: 
[[22432   225]
 [  236  1043]]

Threat: 
[[23883     0]
 [   53     0]]

Insult: 
[[22453   291]
 [  376   816]]

Identity Hate: 
[[23660    49]
 [  164    63]]


In [53]:
len(y_pred_round)

23936

In [11]:

y_pred = model.predict(X_test, verbose=1)

for i in range(len(y_pred)):
    for j in range(6):
        y_pred[i][j] = round(y_pred[i][j])

count = 0
for  i in range(len(y_pred)):
    if (y_test[i]==y_pred[i]).all():
        count +=1
print ("The number of correct predictions: ",count)
print ("The number of attempted predictions: ",len(y_pred))
print ("Accuracy: ", float(count)/len(y_pred))

23936/23936 [==============================] - 28s 1ms/step
('The number of correct predictions: ', 21964)
('The number of attempted predictions: ', 23936)
('Accuracy: ', 0.9176136363636364)


## LSTM + CNN Fully Trained

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dropout,Conv1D, Dense, Input, LSTM, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model,  model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys, os, re, csv, codecs
import numpy as np, pandas as pd
import matplotlib.pyplot as plt


import gensim.models.keyedvectors as word2vec
from sklearn.model_selection import train_test_split
import  h5py

PATH = "/home/ubuntu/fastai/courses/Ait590/"
train_set = pd.read_csv(PATH+"Toxic_PreProc.csv")

list_classes = ["toxic", "severe_toxic", "obscene","threat", "insult", "identity_hate"]
y = train_set[list_classes].values
list_sent_tr = train_set["comment_text"].astype(str)
train_set["comment_text"]=train_set["comment_text"].astype(str)



max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list_sent_tr)
list_sent_tok=tokenizer.texts_to_sequences(list_sent_tr)

max_words=50
Pad_Train=pad_sequences(list_sent_tok,maxlen=max_words)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Pad_Train, y, test_size = 0.15)

inp = Input(shape=(max_words,))

x = Embedding(max_features, 128)(inp)

#Running the bidirectional LSTM model. We are doing 10% regular/recurrent dropout  
x = LSTM(60,return_sequences=True, name="LSTM")(x)

#Adding a CNN layer
Conv_1 = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)


#Adding the maxpooling layer
x = GlobalMaxPool1D()(x)

#passing it through a nonlinear (relu) layer
x = Dense(50, activation="relu")(x)

#One more dropout layer to help with any overfitting
x = Dropout(0.1)(x)

#this is the output layer. we need to use a nonlinear softmax or sigmoid for classification
# Since we have a multi label classification, we'll have to use sigmoid. softmax won't work.
x= Dense(6, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

print (model.summary())

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 128)           2560000   
_________________________________________________________________
LSTM (LSTM)                  (None, 50, 60)            45360     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total para

In [ ]:
batch_size = 32
epochs = 4
hist = model.fit(X_train,y_train, batch_size=batch_size, validation_split=0.1, epochs=epochs)

In [ ]:
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')


In [2]:
y_pred = model.predict(X_test, verbose=1)

for i in range(len(y_pred)):
    for j in range(6):
        y_pred[i][j] = round(y_pred[i][j])

count = 0
for  i in range(len(y_pred)):
    if (y_test[i]==y_pred[i]).all():
        count +=1
print ("The number of correct predictions: ",count)
print ("The number of attempted predictions: ",len(y_pred))
print ("Accuracy: ", float(count)/len(y_pred))


plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')


23936/23936 [==============================] - 14s 588us/step
('The number of correct predictions: ', 21874)
('The number of attempted predictions: ', 23936)
('Accuracy: ', 0.9138536096256684)


## LSTM + CNN Fully Trained (using early_stop)

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dropout,Conv1D, Dense, Input, LSTM, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model,  model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys, os, re, csv, codecs
import numpy as np, pandas as pd
import matplotlib.pyplot as plt


import gensim.models.keyedvectors as word2vec
from sklearn.model_selection import train_test_split
import  h5py

PATH = "/home/ubuntu/fastai/courses/Ait590/"
train_set = pd.read_csv(PATH+"Toxic_PreProc.csv")

list_classes = ["toxic", "severe_toxic", "obscene","threat", "insult", "identity_hate"]
y = train_set[list_classes].values
list_sent_tr = train_set["comment_text"].astype(str)
train_set["comment_text"]=train_set["comment_text"].astype(str)



max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list_sent_tr)
list_sent_tok=tokenizer.texts_to_sequences(list_sent_tr)

max_words=50
Pad_Train=pad_sequences(list_sent_tok,maxlen=max_words)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Pad_Train, y, test_size = 0.15)

inp = Input(shape=(max_words,))

x = Embedding(max_features, 128)(inp)

#Running the bidirectional LSTM model. We are doing 10% regular/recurrent dropout  
x = LSTM(60,return_sequences=True, name="LSTM")(x)

#Adding a CNN layer
Conv_1 = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)


#Adding the maxpooling layer
x = GlobalMaxPool1D()(x)

#passing it through a nonlinear (relu) layer
x = Dense(50, activation="relu")(x)

#One more dropout layer to help with any overfitting
x = Dropout(0.1)(x)

#this is the output layer. we need to use a nonlinear softmax or sigmoid for classification
# Since we have a multi label classification, we'll have to use sigmoid. softmax won't work.
x= Dense(6, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

print (model.summary())

early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

batch_size = 32
epochs = 4
hist = model.fit(X_train,y_train, batch_size=batch_size, validation_split=0.1, epochs=epochs, verbose = 1, callbacks = [early_stop])


##### Saving the model

model_json = model.to_json()

with open("LSTM_CNN_fully.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("LSTM_CNN_fully_weights.h5")

model.save("LSTM_CNN_fully.h5")

y_pred = model.predict(X_test, verbose=1)

for i in range(len(y_pred)):
    for j in range(6):
        y_pred[i][j] = round(y_pred[i][j])

count = 0
for  i in range(len(y_pred)):
    if (y_test[i]==y_pred[i]).all():
        count +=1
print ("The number of correct predictions: ",count)
print ("The number of attempted predictions: ",len(y_pred))
print ("Accuracy: ", float(count)/len(y_pred))


plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')
plt.savefig("test_plot")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 50, 128)           2560000   
_________________________________________________________________
LSTM (LSTM)                  (None, 50, 60)            45360     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 60)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 306       
Total para

## LSTM + CNN + GRU Fully Trained (using early_stop)

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dropout,Conv1D, Dense, Input, LSTM, Activation,  GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model,  model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys, os, re, csv, codecs
import numpy as np, pandas as pd
import matplotlib.pyplot as plt


import gensim.models.keyedvectors as word2vec
from sklearn.model_selection import train_test_split
import  h5py

PATH = "/home/ubuntu/fastai/courses/Ait590/"
train_set = pd.read_csv(PATH+"Toxic_PreProc.csv")

list_classes = ["toxic", "severe_toxic", "obscene","threat", "insult", "identity_hate"]
y = train_set[list_classes].values
list_sent_tr = train_set["comment_text"].astype(str)
train_set["comment_text"]=train_set["comment_text"].astype(str)



max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list_sent_tr)
list_sent_tok=tokenizer.texts_to_sequences(list_sent_tr)

max_words=50
Pad_Train=pad_sequences(list_sent_tok,maxlen=max_words)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Pad_Train, y, test_size = 0.15, random_state=42)

inp = Input(shape=(max_words,))

x = Embedding(max_features, 128)(inp)

#Running the bidirectional LSTM model. We are doing 10% regular/recurrent dropout  
x = LSTM(60,return_sequences=True, name="LSTM")(x)

#GRU
x = Bidirectional(GRU(128, return_sequences = True))(x)

#Adding a CNN layer
Conv_1 = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)


#Adding the maxpooling layer
x = GlobalMaxPool1D()(x)

#passing it through a nonlinear (relu) layer
x = Dense(50, activation="relu")(x)

#One more dropout layer to help with any overfitting
x = Dropout(0.1)(x)

#this is the output layer. we need to use a nonlinear softmax or sigmoid for classification
# Since we have a multi label classification, we'll have to use sigmoid. softmax won't work.
x= Dense(6, activation="sigmoid")(x)

model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

print (model.summary())

early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

batch_size = 32
epochs = 4
hist = model.fit(X_train,y_train, batch_size=batch_size, validation_split=0.1, epochs=epochs, verbose = 1, callbacks = [early_stop])


##### Saving the model

model_json = model.to_json()

with open("LSTM_CNN_fully.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("LSTM_CNN_fully_weights.h5")

model.save("LSTM_CNN_fully.h5")

y_pred = model.predict(X_test, verbose=1)

for i in range(len(y_pred)):
    for j in range(6):
        y_pred[i][j] = round(y_pred[i][j])

count = 0
for  i in range(len(y_pred)):
    if (y_test[i]==y_pred[i]).all():
        count +=1
print ("The number of correct predictions: ",count)
print ("The number of attempted predictions: ",len(y_pred))
print ("Accuracy: ", float(count)/len(y_pred))


plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')
plt.savefig("test_plot")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 50, 128)           2560000   
_________________________________________________________________
LSTM (LSTM)                  (None, 50, 60)            45360     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 256)           145152    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                12850     
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
__________

## LSTM Word2vec

In [3]:
word2vec_dict = word2vec.KeyedVectors.load_word2vec_format("/home/ubuntu/fastai/courses/Ait590/Embedding/GoogleNews-vectors-negative300.bin"
                                                          , binary=True)
pretrain_emb = dict()
for tok in word2vec_dict.wv.vocab:
    pretrain_emb[tok] = word2vec_dict.word_vec(tok)
all_embs=np.stack(list(pretrain_emb.values()))
Gl_emb_mean, Gl_Std=all_embs.mean(), all_embs.std()
num_words=len(tokenizer.word_index)
embed_size=300
pretrain_matrix = np.random.normal(Gl_emb_mean, Gl_Std, (num_words, embed_size))

count = 0
for tok , i in tokenizer.word_index.items():
    
    if pretrain_emb.get(tok) is None:
        continue
    
    pretrain_matrix[i-1] = pretrain_emb[tok] 
    count +=1
print count

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


65522


In [5]:
inp = Input(shape=(max_words,))
x = Embedding(pretrain_matrix.shape[0], pretrain_matrix.shape[1],trainable=False,weights=[pretrain_matrix])(inp)
x = LSTM(60,return_sequences=True,dropout=.1,recurrent_dropout=.1,name="BI_LSTM")(x)
#Adding the maxpooling layer
x = GlobalMaxPool1D()(x)
#One more dropout layer to help with any overfitting
x = Dropout(0.2)(x)
#passing it through a nonlinear (relu) layer
x = Dense(50, activation="relu")(x)
#One more dropout layer to help with any overfitting
x = Dropout(0.2)(x)
#this is the output layer. we need to use a nonlinear softmax or sigmoid for classification
# Since we have a multi label classification, we'll have to use sigmoid. softmax won't work.
x= Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')
model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[168663,300]
	 [[Node: embedding_2/embeddings/Assign = Assign[T=DT_FLOAT, _class=["loc:@embedding_2/embeddings"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_2/embeddings, embedding_2/random_uniform)]]

Caused by op u'embedding_2/embeddings/Assign', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-p

In [ ]:
batch_size = 32
epochs = 2
log = model.fit(X_train,y_train, batch_size=batch_size, validation_split=0.1, epochs=epochs)

In [ ]:
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(log.history['loss'], color='b', label='Training Loss')
plt.plot(log.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')
#plt.save("Bi-LSTM-Word2vec-train-valid-loss")

In [ ]:
import h5py
model.save('word2vec_BI_lstm_model.h5')
model_json = model.to_json()
with open("word2vec_BI_lstm_model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("word2vec_BI_lstm_model_weights.h5")
model=load_model('word2vec_BI_lstm_model.h5')
y_pred = model.predict(X_test, verbose=1)

for i in range(len(y_pred)):
    for j in range(6):
        y_pred[i][j] = round(y_pred[i][j])

count = 0
for  i in range(len(y_pred)):
    if (y_test[i]==y_pred[i]).all():
        count +=1
print ("The number of correct predictions: ",count)
print ("The number of attempted predictions: ",len(y_pred))
print ("Accuracy: ", float(count)/len(y_pred))

In [ ]:
print("Toxic:\n{}".format(confusion_matrix(y_test[:,0], y_pred[:,0])))
print("\nSevere Tox: \n    {}".format(confusion_matrix(y_test[:,1], y_pred[:,1])))
print("\nObscene: \n    {}".format(confusion_matrix(y_test[:,2], y_pred[:,2])))
print("\nThreat: \n     {}".format(confusion_matrix(y_test[:,3], y_pred[:,3])))
print("\nInsult: \n    {}".format(confusion_matrix(y_test[:,4], y_pred[:,4])))
print("\nIden Hate: \n    {}".format(confusion_matrix(y_test[:,5], y_pred[:,5])))